In [2]:
!pip install ArabertPreprocessor

ERROR: Could not find a version that satisfies the requirement ArabertPreprocessor (from versions: none)
ERROR: No matching distribution found for ArabertPreprocessor

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# finetune process 
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np


In [5]:
# load the data
bbc_data = pd.read_csv('bbc_news_arabic_summarization.csv')
# Randomly sample 50% of the dataset
#bbc_data = bbc_data.sample(frac=0.5, random_state=42)

In [6]:
bbc_data.head()

,id,url,title,summary,text
0,140323_russian_troops_crimea_naval_base,https://www.bbc.com/arabic/worldnews/2014/03/1...,القوات الأوكرانية تبدأ الانسحاب من القرم,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينو...
1,130528_egypt_nile_dam,https://www.bbc.com/arabic/middleeast/2013/05/...,هل يفرض سد النهضة الإثيوبي واقعا جديدا على مصر؟,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...",بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...
2,world-47242349,https://www.bbc.com/arabic/world-47242349,تعرف على منطقة كشمير التي تسببت بحربين بين اله...,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...
3,vert-cul-55078328,https://www.bbc.com/arabic/vert-cul-55078328,ماذا تعرف عن العالم الخفي للمعابد اليابانية ال...,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...
4,141023_yemen_hodeida,https://www.bbc.com/arabic/middleeast/2014/10/...,"اشتباك بين الحوثيين و""الحراك التهامي"" في الحدي...","أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...


In [7]:
bbc_data.dropna(subset=['text', 'summary'], inplace=True)
bbc_data = bbc_data.drop(columns=['id','url','title','summary'])

In [8]:
from datasets import Dataset
bbc_dataset= Dataset.from_pandas(bbc_data)
train_test_split = bbc_dataset.train_test_split(test_size=0.2, seed=42)
bbc_train_dataset1 = train_test_split['train']
bbc_test_dataset = train_test_split['test']
# Split the training set further into 80% train and 20% validation
bbc_train_dataset, bbc_val_dataset = bbc_train_dataset1.train_test_split(test_size=0.2, seed=42).values()



/Users/jameelamer/MasterProject/Summarization/summrization_webapp/flaskProject2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from arabert.preprocess import ArabertPreprocessor

# Initialize AraBERT preprocessor
arabert_prep = ArabertPreprocessor("aubmindlab/aragpt2-base")
# Preprocess dataset for Arabic
def preprocess_arabic(examples):
    examples["text"] = [arabert_prep.preprocess(text) for text in examples["text"]]
    return examples


train_dataset = bbc_train_dataset.map(preprocess_arabic, batched=True)
val_dataset = bbc_val_dataset.map(preprocess_arabic, batched=True)
test_dataset = bbc_test_dataset.map(preprocess_arabic, batched=True)
# Load GPT-2 model and tokenizer for Arabic
model_name = "aubmindlab/aragpt2-base"  # Pre-trained Arabic GPT-2
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)





Map: 100%|██████████| 9380/9380 [00:10<00:00, 861.21 examples/s]


In [14]:
print(train_dataset.column_names)

['text']


In [15]:
# Tokenize dataset
# Set the pad token to <|endoftext|>
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

#tokenized_datasets = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])   "__index_level_0__"
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal language modeling
)
print(train_dataset.column_names)


Map: 100%|██████████| 7504/7504 [00:17<00:00, 439.25 examples/s]

['input_ids', 'attention_mask']


In [20]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./SSS_arabic_gpt2_finetuned",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,  # Adjust for CPU
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    save_strategy="epoch",
    use_cpu=True,
    remove_unused_columns=False,  # Prevent Trainer from filtering columns
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)



ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [15]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.275600,3.719555
2,4.035000,3.648158
3,3.894000,3.626880


TrainOutput(global_step=11256, training_loss=4.154550298868381, metrics={'train_runtime': 51434.1435, 'train_samples_per_second': 0.875, 'train_steps_per_second': 0.219, 'total_flos': 1.1762844696576e+16, 'train_loss': 4.154550298868381, 'epoch': 3.0})

In [1]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./arabic_gpt2_finetuned")
tokenizer.save_pretrained("./arabic_gpt2_finetuned")

NameError: name 'model' is not defined

In [4]:
input_text = "قطر"
input_ids = tokenizer.encode(input_text, return_tensors="pt", padding=True).to("cpu")

# Generate text
output = model.generate(
    input_ids,
    max_length=200,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

# Decode the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)

NameError: name 'tokenizer' is not defined

In [20]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./smart_soft_arabic_gpt2")
tokenizer.save_pretrained("./smart_soft_arabic_gpt2")

('./smart_soft_arabic_gpt2/tokenizer_config.json',
 './smart_soft_arabic_gpt2/special_tokens_map.json',
 './smart_soft_arabic_gpt2/vocab.json',
 './smart_soft_arabic_gpt2/merges.txt',
 './smart_soft_arabic_gpt2/added_tokens.json')

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from arabert.preprocess import ArabertPreprocessor


/Users/jameelamer/MasterProject/Summarization/summrization_webapp/flaskProject2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Load GPT-2 model and tokenizer for Arabic
model_name = "./sss-arabic_gpt2_finetuned"  # Pre-trained Arabic GPT-2
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [4]:
input_text ="الحرب السورية"
input_ids = tokenizer.encode(input_text, return_tensors="pt", padding=True).to("cpu")

# Generate text
output = model.generate(
    input_ids,
    max_length=200,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

# Decode the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text:
الحرب السورية مستمرة منذ أكثر من ثلاث سنوات وقال المرصد السوري لحقوق الإنسان المعارض ، ومقره بريطانيا ، إن الغارات الجوية التي يشنها التحالف الدولي بقيادة الولايات المتحدة على مواقع تنظيم " الدولة الإسلامية " في محافظة دير الزور أسفرت عن مقتل العشرات من مسلحي التنظيم . وأضاف المرصد أن الغارات أسفرت أيضا عن تدمير عدد من المنازل في المنطقة . وقال رامي عبد الرحمن مدير المرصد ، الذي يتخذ من بريطانيا مقرا له ، لبي بي سي إن الضربات الجوية استهدفت مواقع للتنظيم في مدينة البوكمال ، شرقي سوريا . مواضيع قد تهمك نهاية وأضاف أن طائرات التحالف شنت أيضا غارات على مناطق أخرى في سوريا ، بما في ذلك مدينة الرقة ، التي يسيطر عليها مسلحو التنظيم ، بحسب المرصد . وأشار المرصد إلى أن غارات التحالف استهدفت أيضا مواقع لمسلحي التنظيم في الرقة ودير الزور . وكانت القوات الحكومية السورية ، مدعومة بغطاء جوي روسي ، قد شنت في وقت سابق من الشهر الجاري ، عملية عسكرية لاستعادة السيطرة على مدينة تدمر الأثرية ، الواقعة على بعد 50 كيلومترا غرب العاصمة السورية دمشق ، والتي تعد من أهم المواقع الأثرية في العال